In [3]:
import pandas as pd

In [4]:
Datasets_path="C:\\Users\\asus\\Desktop\\Data Science\\Machine Learning\\Project\\Datasets"
Data=pd.read_csv(f"{Datasets_path}\\delays_train.csv")

In [5]:
Data.isna().sum()

Weekday                           0
Month_of_Year                     0
Day_of_Month                      0
Scheduled_Departure_Time          0
Scheduled_Arrival_Time            0
Marketing_Airline            163551
Marketing_Airline_DOT_ID          0
Flight_Number                     0
Origin_Airport_ID                 0
Destination_Airport_ID       163094
Flight_Cancelled                  0
Departure_State                   0
Arrival_State                     0
Departure_Delay              206461
Arrival_Delay                211306
Diverted_Airport_Landings         0
Taxi_Out_Time                 49225
Taxi_In_Time                  49867
Flight_Diverted                   0
Actual_Departure_Time         48326
Flight_Duration              211490
Flight_Distance              163190
Origin_Temperature           163190
Destination_Temperature      163190
Origin_Wind_Speed            211490
Destination_Wind_Speed       211490
Origin_Precipitation              0
Destination_Precipitation   

In [13]:
Data.shape

(838241, 25)

In [6]:
Data.dropna(inplace=True)

In [5]:
Data.columns

Index(['Weekday', 'Month_of_Year', 'Day_of_Month', 'Scheduled_Departure_Time',
       'Scheduled_Arrival_Time', 'Marketing_Airline',
       'Marketing_Airline_DOT_ID', 'Flight_Number', 'Origin_Airport_ID',
       'Destination_Airport_ID', 'Flight_Cancelled', 'Departure_State',
       'Arrival_State', 'Departure_Delay', 'Arrival_Delay',
       'Diverted_Airport_Landings', 'Taxi_Out_Time', 'Taxi_In_Time',
       'Flight_Diverted', 'Actual_Departure_Time', 'Flight_Duration',
       'Flight_Distance', 'Origin_Temperature', 'Destination_Temperature',
       'Origin_Wind_Speed', 'Destination_Wind_Speed', 'Origin_Precipitation',
       'Destination_Precipitation'],
      dtype='object')

In [34]:
Data.dtypes

Weekday                        int64
Month_of_Year                  int64
Day_of_Month                   int64
Scheduled_Departure_Time       int64
Scheduled_Arrival_Time         int64
Marketing_Airline_DOT_ID       int64
Flight_Number                  int64
Origin_Airport_ID              int64
Destination_Airport_ID       float64
Flight_Cancelled                bool
Departure_Delay              float64
Arrival_Delay                float64
Diverted_Airport_Landings      int64
Taxi_Out_Time                float64
Taxi_In_Time                 float64
Flight_Diverted                 bool
Actual_Departure_Time        float64
Flight_Duration              float64
Flight_Distance              float64
Origin_Temperature           float64
Destination_Temperature      float64
Origin_Wind_Speed            float64
Destination_Wind_Speed       float64
Origin_Precipitation         float64
Destination_Precipitation    float64
dtype: object

In [22]:
# columns_to_drop = ['Arrival_State', 'Departure_State', 'Marketing_Airline']
# Data.drop(columns=columns_to_drop, inplace=True)

In [7]:
Data = pd.get_dummies(Data, columns=['Arrival_State'], drop_first=True)
Data = pd.get_dummies(Data, columns=['Departure_State'], drop_first=True)
Data = pd.get_dummies(Data, columns=['Marketing_Airline'], drop_first=True)

### Grid Search for CV

### Outlier Removed

In [9]:
import numpy as np
numeric_columns = Data.select_dtypes(include=[np.number]).columns
Q1 = Data[numeric_columns].quantile(0.25)
Q3 = Data[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
# Define the threshold for outliers
threshold = 1.5
# Filter out rows with outliers in any numeric column
Data_no_outliers = Data[~((Data[numeric_columns] < (Q1 - threshold * IQR)) | (Data[numeric_columns] > (Q3 + threshold * IQR))).any(axis=1)]

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

# Assuming Data_no_outliers is your preprocessed DataFrame
# X contains features, y contains target variable
X = Data_no_outliers.drop(columns=['Arrival_Delay'])
y = Data_no_outliers['Arrival_Delay']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}

# Set up the GridSearchCV
grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model to the data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best MSE (from cross-validation): {best_score}')

# Fit the Lasso regression model with the best parameters
best_lasso = grid_search.best_estimator_

# Predict on the test set
y_pred = best_lasso.predict(X_test_scaled)

# Calculate RMSE, MAE, and MAPE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Root Mean Squared Error (RMSE) on test set: {rmse}')
print(f'Mean Absolute Error (MAE) on test set: {mae}')
print(f'Mean Absolute Percentage Error (MAPE) on test set: {mape}')

Best parameters: {'alpha': 0.1}
Best MSE (from cross-validation): 48.63137219554509
Root Mean Squared Error (RMSE) on test set: 6.935155690634248
Mean Absolute Error (MAE) on test set: 5.305965713450897
Mean Absolute Percentage Error (MAPE) on test set: 5.266880767952744


### Outlier Imputed With Winsorization 

In [44]:
import numpy as np
numeric_columns = Data.select_dtypes(include=[np.number]).columns
Q1 = Data[numeric_columns].quantile(0.25)
Q3 = Data[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
threshold = 1.5
lower_limit = Q1 - threshold * IQR
upper_limit = Q3 + threshold * IQR
Data_imputed = Data.copy()
for column in numeric_columns:
    lower_outliers_mask = Data_imputed[column] < lower_limit[column]
    Data_imputed.loc[lower_outliers_mask, column] = lower_limit[column]
for column in numeric_columns:
    upper_outliers_mask = Data_imputed[column] > upper_limit[column]
    Data_imputed.loc[upper_outliers_mask, column] = upper_limit[column]

C:\Users\asus\AppData\Local\Temp\ipykernel_20976\3431522854.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-14.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_imputed.loc[lower_outliers_mask, column] = lower_limit[column]
C:\Users\asus\AppData\Local\Temp\ipykernel_20976\3431522854.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-314.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_imputed.loc[lower_outliers_mask, column] = lower_limit[column]
C:\Users\asus\AppData\Local\Temp\ipykernel_20976\3431522854.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-123.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_imputed.loc[lo

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

# Assuming Data_no_outliers is your preprocessed DataFrame
# X contains features, y contains target variable
X = Data_imputed.drop(columns=['Arrival_Delay'])
y = Data_imputed['Arrival_Delay']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}

# Set up the GridSearchCV
grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model to the data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best MSE (from cross-validation): {best_score}')

# Fit the Lasso regression model with the best parameters
best_lasso = grid_search.best_estimator_

# Predict on the test set
y_pred = best_lasso.predict(X_test_scaled)

# Calculate RMSE, MAE, and MAPE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Root Mean Squared Error (RMSE) on test set: {rmse}')
print(f'Mean Absolute Error (MAE) on test set: {mae}')
print(f'Mean Absolute Percentage Error (MAPE) on test set: {mape}')

Best parameters: {'alpha': 0.1}
Best MSE (from cross-validation): 94.61102802416546
Root Mean Squared Error (RMSE) on test set: 9.731371391099657
Mean Absolute Error (MAE) on test set: 7.260942914101785
Mean Absolute Percentage Error (MAPE) on test set: 7.8795392757185585


### Feature Selection
***Model only with the Significant Features***

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# Assuming X contains your features including all features, and y contains your target variable (Arrival_Delay)
X = Data_no_outliers.drop(columns=['Arrival_Delay'])  # Assuming 'Arrival_Delay' is your target variable
y = Data_no_outliers['Arrival_Delay']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Normalizing all features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fitting the Elastic Net model
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)  # You can adjust alpha and l1_ratio as needed
elastic_net.fit(X_train_scaled, y_train)

# Extracting coefficients to identify important features
feature_importance = elastic_net.coef_

# Printing feature importance (coefficients)
print("Feature Importance (Coefficients):")
for i, feature in enumerate(X.columns):
    print(f"{feature}: {feature_importance[i]}")

# Making predictions
y_pred = elastic_net.predict(X_test_scaled)

# Calculating MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculating MSE
mse = mean_squared_error(y_test, y_pred)

print("\nMAPE:", mape)
print("MSE:", mse)

Feature Importance (Coefficients):
Weekday: 0.0
Month_of_Year: 0.26745574715810017
Day_of_Month: -0.0
Scheduled_Departure_Time: 0.0
Scheduled_Arrival_Time: 0.0
Marketing_Airline_DOT_ID: -0.0
Flight_Number: -0.12750878323398407
Origin_Airport_ID: -0.0
Destination_Airport_ID: 0.0
Flight_Cancelled: 0.0
Departure_Delay: 5.344322497859857
Diverted_Airport_Landings: 0.0
Taxi_Out_Time: 2.155429840647149
Taxi_In_Time: 0.6074069192906147
Flight_Diverted: 0.0
Actual_Departure_Time: 0.061014972333126034
Flight_Duration: 0.0
Flight_Distance: -0.1918515514366262
Origin_Temperature: -0.0
Destination_Temperature: -0.0
Origin_Wind_Speed: 0.0
Destination_Wind_Speed: 0.0
Origin_Precipitation: 0.0
Destination_Precipitation: 0.0
Arrival_State_AL: 0.0
Arrival_State_AR: 0.0
Arrival_State_AZ: 0.0
Arrival_State_CA: -0.0
Arrival_State_CO: 0.0
Arrival_State_CT: 0.0
Arrival_State_DE: 0.0
Arrival_State_FL: 0.0
Arrival_State_GA: 0.0
Arrival_State_HI: 0.18330275687935038
Arrival_State_IA: -0.0
Arrival_State_ID: 0.0

# FINAL MODEL !!!

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

# Assuming Data_no_outliers is your preprocessed DataFrame
# X contains features, y contains target variable
X = Data_no_outliers.drop(columns=['Arrival_Delay'])
y = Data_no_outliers['Arrival_Delay']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}

# Set up the GridSearchCV
lasso_grid_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit Lasso to perform feature selection
lasso_grid_search.fit(X_train_scaled, y_train)

# Get the best Lasso model
best_lasso = lasso_grid_search.best_estimator_

# Get the indices of non-zero coefficients
significant_indices = np.where(best_lasso.coef_ != 0)[0]

# Get the names of the significant features
significant_features = X.columns[significant_indices]
print(f'Significant features: {significant_features}')

# Select only the significant features from X_train_scaled and X_test_scaled
X_train_scaled_significant = X_train_scaled[:, significant_indices]
X_test_scaled_significant = X_test_scaled[:, significant_indices]

# Define the parameter grid for Lasso after feature selection
lasso_param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}

# Set up the GridSearchCV for Lasso after feature selection
lasso_grid_search_significant = GridSearchCV(Lasso(), lasso_param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit Lasso again using only significant features
lasso_grid_search_significant.fit(X_train_scaled_significant, y_train)

# Get the best parameters and best score for Lasso after feature selection
best_params_significant = lasso_grid_search_significant.best_params_
best_score_significant = -lasso_grid_search_significant.best_score_

print(f'Best parameters after feature selection: {best_params_significant}')
print(f'Best MSE (from cross-validation) after feature selection: {best_score_significant}')

# Fit the Lasso regression model with the best parameters using only significant features
best_lasso_significant = lasso_grid_search_significant.best_estimator_

# Predict on the test set using only significant features
y_pred_significant = best_lasso_significant.predict(X_test_scaled_significant)

# Calculate MAPE for significant features
mape_significant = mean_absolute_percentage_error(y_test, y_pred_significant)

# Calculate RMSE for significant features
rmse_significant = mean_squared_error(y_test, y_pred_significant, squared=False)

# Calculate MAE for significant features
mae_significant = mean_absolute_error(y_test, y_pred_significant)

print(f'Mean Absolute Percentage Error (MAPE) on test set using significant features: {mape_significant}')
print(f'Root Mean Squared Error (RMSE) on test set using significant features: {rmse_significant}')
print(f'Mean Absolute Error (MAE) on test set using significant features: {mae_significant}')

Significant features: Index(['Month_of_Year', 'Scheduled_Departure_Time', 'Flight_Number',
       'Departure_Delay', 'Taxi_Out_Time', 'Taxi_In_Time', 'Flight_Duration',
       'Flight_Distance', 'Arrival_State_AL', 'Arrival_State_AR',
       'Arrival_State_AZ', 'Arrival_State_CA', 'Arrival_State_CO',
       'Arrival_State_CT', 'Arrival_State_FL', 'Arrival_State_GA',
       'Arrival_State_HI', 'Arrival_State_ID', 'Arrival_State_IL',
       'Arrival_State_MA', 'Arrival_State_ME', 'Arrival_State_MI',
       'Arrival_State_MO', 'Arrival_State_MT', 'Arrival_State_NC',
       'Arrival_State_NJ', 'Arrival_State_NV', 'Arrival_State_NY',
       'Arrival_State_OH', 'Arrival_State_OR', 'Arrival_State_PA',
       'Arrival_State_PR', 'Arrival_State_RI', 'Arrival_State_SC',
       'Arrival_State_TT', 'Arrival_State_TX', 'Arrival_State_UT',
       'Arrival_State_VA', 'Arrival_State_VI', 'Arrival_State_VT',
       'Arrival_State_WA', 'Departure_State_AL', 'Departure_State_AZ',
       'Departure_State_

## Test  Data set 

### Prediction only for the Non-Missing Values

In [221]:
Datasets_path="C:\\Users\\asus\\Desktop\\Data Science\\Machine Learning\\Project\\Datasets"
Data_test=pd.read_csv(f"{Datasets_path}\\delays_test.csv")

In [222]:
Data_test.isna().sum()

Weekday                          0
Month_of_Year                    0
Day_of_Month                     0
Scheduled_Departure_Time         0
Scheduled_Arrival_Time           0
Marketing_Airline            40364
Marketing_Airline_DOT_ID         0
Flight_Number                    0
Origin_Airport_ID                0
Destination_Airport_ID       40821
Flight_Cancelled                 0
Departure_State                  0
Arrival_State                    0
Departure_Delay              51789
Diverted_Airport_Landings        0
Taxi_Out_Time                12264
Taxi_In_Time                 12430
Flight_Diverted                  0
Actual_Departure_Time        12016
Flight_Duration              52666
Flight_Distance              40725
Origin_Temperature           40725
Destination_Temperature      40725
Origin_Wind_Speed            52666
Destination_Wind_Speed       52666
Origin_Precipitation             0
Destination_Precipitation        0
dtype: int64

In [223]:
Data_test.dropna(inplace=True)

In [224]:
Data_test = pd.get_dummies(Data_test, columns=['Arrival_State'], drop_first=True)
Data_test = pd.get_dummies(Data_test, columns=['Departure_State'], drop_first=True)
Data_test = pd.get_dummies(Data_test, columns=['Marketing_Airline'], drop_first=True)

In [215]:
# import numpy as np
# numeric_columns = Data_test.select_dtypes(include=[np.number]).columns
# Q1 = Data_test[numeric_columns].quantile(0.25)
# Q3 = Data_test[numeric_columns].quantile(0.75)
# IQR = Q3 - Q1
# # Define the threshold for outliers
# threshold = 1.5
# # Filter out rows with outliers in any numeric column
# Data_no_outliers = Data_test[~((Data_test[numeric_columns] < (Q1 - threshold * IQR)) | (Data_test[numeric_columns] > (Q3 + threshold * IQR))).any(axis=1)]

### Outlier Handling
***Winsorization***

In [226]:
import numpy as np
numeric_columns = Data_test.select_dtypes(include=[np.number]).columns
Q1 = Data_test[numeric_columns].quantile(0.25)
Q3 = Data_test[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
threshold = 1.5
lower_limit = Q1 - threshold * IQR
upper_limit = Q3 + threshold * IQR
Data_test_outlier_Imputed = Data_test.copy()
for column in numeric_columns:
    lower_outliers_mask = Data_test_outlier_Imputed[column] < lower_limit[column]
    Data_test_outlier_Imputed.loc[lower_outliers_mask, column] = lower_limit[column]
for column in numeric_columns:
    upper_outliers_mask = Data_test_outlier_Imputed[column] > upper_limit[column]
    Data_test_outlier_Imputed.loc[upper_outliers_mask, column] = upper_limit[column]

C:\Users\asus\AppData\Local\Temp\ipykernel_16304\1493213730.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-14.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_test_outlier_Imputed.loc[lower_outliers_mask, column] = lower_limit[column]
C:\Users\asus\AppData\Local\Temp\ipykernel_16304\1493213730.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-119.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_test_outlier_Imputed.loc[lower_outliers_mask, column] = lower_limit[column]
C:\Users\asus\AppData\Local\Temp\ipykernel_16304\1493213730.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '19509.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype f

In [16]:
Data_test.isna().sum()

Weekday                     0
Month_of_Year               0
Day_of_Month                0
Scheduled_Departure_Time    0
Scheduled_Arrival_Time      0
                           ..
Marketing_Airline_G4        0
Marketing_Airline_HA        0
Marketing_Airline_NK        0
Marketing_Airline_UA        0
Marketing_Airline_WN        0
Length: 137, dtype: int64

### Prediction 

In [228]:
# Assuming new_test_data is your new dataset without the target variable
new_test_data_scaled = scaler.transform(Data_test_outlier_Imputed)

# Select only the significant features
new_test_data_scaled_significant = new_test_data_scaled[:, significant_indices]

# Predict the target variable using the trained model with significant features
predictions = best_lasso_significant.predict(new_test_data_scaled_significant)

# Create a DataFrame to store predictions with their indices
predictions_df = pd.DataFrame({
    'Index': Data_test_outlier_Imputed.index,
    'Predicted_Arrival_Delay': predictions
})

# Save the predictions to a CSV file at the specified path
predictions_file_path = os.path.join(Datasets_path, "Prediction_for_Non_Missing_Observations.csv")
predictions_df.to_csv(predictions_file_path, index=False)

print(f"Predictions saved to: {predictions_file_path}")

Predictions saved to: C:\Users\asus\Desktop\Data Science\Machine Learning\Project\Datasets\Prediction_for_Non_Missing_Observations.csv


### Prediction for the whole Data

In [184]:
Datasets_path="C:\\Users\\asus\\Desktop\\Data Science\\Machine Learning\\Project\\Datasets"
Data_test_whole=pd.read_csv(f"{Datasets_path}\\delays_test.csv")

In [185]:
Data_test_whole.isna().sum()

Weekday                          0
Month_of_Year                    0
Day_of_Month                     0
Scheduled_Departure_Time         0
Scheduled_Arrival_Time           0
Marketing_Airline            40364
Marketing_Airline_DOT_ID         0
Flight_Number                    0
Origin_Airport_ID                0
Destination_Airport_ID       40821
Flight_Cancelled                 0
Departure_State                  0
Arrival_State                    0
Departure_Delay              51789
Diverted_Airport_Landings        0
Taxi_Out_Time                12264
Taxi_In_Time                 12430
Flight_Diverted                  0
Actual_Departure_Time        12016
Flight_Duration              52666
Flight_Distance              40725
Origin_Temperature           40725
Destination_Temperature      40725
Origin_Wind_Speed            52666
Destination_Wind_Speed       52666
Origin_Precipitation             0
Destination_Precipitation        0
dtype: int64

### Feature Engineering

In [186]:
Data_test_whole['Marketing_Airline'] = Data_test_whole.groupby('Marketing_Airline_DOT_ID')['Marketing_Airline'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))

In [187]:
# Group by 'Scheduled_Departure_Time' and fill missing values of 'Departure_Delay' with mean
Data_test_whole['Departure_Delay'] = Data_test_whole.groupby('Scheduled_Departure_Time')['Departure_Delay'].transform(lambda x: x.fillna(x.median()))

In [188]:
Data_test_whole['Departure_Delay'].fillna(Data_test_whole['Departure_Delay'].mean(), inplace=True)

In [189]:
Data_test_whole.Taxi_In_Time.isna().sum()

12430

In [190]:
# Group by 'Scheduled_Departure_Time' and fill missing values of 'Departure_Delay' with mean
Data_test_whole['Taxi_Out_Time'] = Data_test_whole.groupby('Origin_Airport_ID')['Taxi_Out_Time'].transform(lambda x: x.fillna(x.median()))

In [191]:
Data_test_whole['Taxi_Out_Time'].fillna(Data_test_whole['Taxi_Out_Time'].mean(), inplace=True)

In [192]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Select relevant features
relevant_features = ['Marketing_Airline_DOT_ID', 'Taxi_In_Time']

# Group by 'Destination_Airport_ID' and impute missing values of 'Taxi_In_Time' by median within each group
grouped = Data_test_whole.groupby(['Marketing_Airline_DOT_ID'])
imputer = SimpleImputer(strategy='median')

# Impute missing values for 'Taxi_In_Time' within each group
Data_test_whole['Taxi_In_Time'] = grouped['Taxi_In_Time'].transform(lambda x: imputer.fit_transform(x.values.reshape(-1, 1)).flatten())


In [199]:
Data_test_whole.isna().sum()

Weekday                     0
Month_of_Year               0
Day_of_Month                0
Scheduled_Departure_Time    0
Scheduled_Arrival_Time      0
                           ..
Marketing_Airline_G4        0
Marketing_Airline_HA        0
Marketing_Airline_NK        0
Marketing_Airline_UA        0
Marketing_Airline_WN        0
Length: 137, dtype: int64

In [194]:
# Group by 'Scheduled_Departure_Time' and fill missing values of 'Departure_Delay' with mean
Data_test_whole['Flight_Duration'] = Data_test_whole.groupby('Origin_Airport_ID')['Flight_Duration'].transform(lambda x: x.fillna(x.median()))

In [195]:
Data_test_whole['Flight_Duration'].fillna(Data_test_whole['Flight_Duration'].mean(), inplace=True)

In [196]:
Data_test_whole['Flight_Distance'] = Data_test_whole.groupby('Origin_Airport_ID')['Flight_Distance'].transform(lambda x: x.fillna(x.median()))

In [198]:
Data_test_whole = pd.get_dummies(Data_test_whole, columns=['Arrival_State'], drop_first=True)
Data_test_whole = pd.get_dummies(Data_test_whole, columns=['Departure_State'], drop_first=True)
Data_test_whole = pd.get_dummies(Data_test_whole, columns=['Marketing_Airline'], drop_first=True)

### Handling Outliers
***Imputing***

In [235]:
# import numpy as np
# numeric_columns = Data.select_dtypes(include=[np.number]).columns
# Q1 = Data[numeric_columns].quantile(0.25)
# Q3 = Data[numeric_columns].quantile(0.75)
# IQR = Q3 - Q1
# # Define the threshold for outliers
# threshold = 1.5
# # Filter out rows with outliers in any numeric column
# Data = Data[~((Data[numeric_columns] < (Q1 - threshold * IQR)) | (Data[numeric_columns] > (Q3 + threshold * IQR))).any(axis=1)]

In [200]:
import numpy as np
numeric_columns = Data_test_whole_Sig.select_dtypes(include=[np.number]).columns
Q1 = Data_test_whole_Sig[numeric_columns].quantile(0.25)
Q3 = Data_test_whole_Sig[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
threshold = 1.5
lower_limit = Q1 - threshold * IQR
upper_limit = Q3 + threshold * IQR
Data_test_outlier_Imputed = Data_test_whole.copy()
for column in numeric_columns:
    lower_outliers_mask = Data_test_outlier_Imputed[column] < lower_limit[column]
    Data_test_outlier_Imputed.loc[lower_outliers_mask, column] = lower_limit[column]
for column in numeric_columns:
    upper_outliers_mask = abdi[column] > upper_limit[column]
    Data_test_outlier_Imputed.loc[upper_outliers_mask, column] = upper_limit[column]

C:\Users\asus\AppData\Local\Temp\ipykernel_16304\3721539595.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-3067.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Data_test_outlier_Imputed.loc[lower_outliers_mask, column] = lower_limit[column]


### Prediction 

In [211]:
import pandas as pd

# Scale the test data using the same scaler fitted on the entire training dataset
Data_test_imputed_scaled = scaler.transform(Data_test_outlier_Imputed)

# Select only the significant features from the scaled test data
new_test_data_scaled_significant = Data_test_imputed_scaled[:, significant_indices]

# Predict the target variable using the model fitted with only significant features
predictions = best_lasso_significant.predict(new_test_data_scaled_significant)

# Create a DataFrame to store predictions with their indices
predictions_df = pd.DataFrame({
    'Index': Data_test_outlier_Imputed.index,  # Assuming the index is preserved
    'Predicted_Arrival_Delay': predictions
})

# Save the predictions to a CSV file at the specified path
predictions_file_path = os.path.join(Datasets_path, "Predictions_for_all_test_data.cs")
predictions_df.to_csv(predictions_file_path, index=False)

print(f"Predictions saved to: {predictions_file_path}")

Predictions saved to: C:\Users\asus\Desktop\Data Science\Machine Learning\Project\Datasets\Predictions_for_all_test_data.csv
